Fabian Gonzalez Vera | A01367585

In [1]:
#industria 

In [2]:
import pandas as pd
import numpy as np

In [3]:
dataf = pd.read_csv('dataus2023.csv')
firmf = pd.read_csv('firmsus2023.csv')
print(dataf.shape, firmf.shape)

(533088, 21) (5615, 7)


merge industry with data

In [5]:
dataf['qdate'] = pd.PeriodIndex(dataf.q, freq='Q')

In [13]:
firmf.columns

Index(['empresa', 'Nombre', 'status', 'partind', 'naics1', 'naics2',
       'SectorEconomatica'],
      dtype='object')

In [16]:
firms = firmf[['empresa','Nombre','status','naics1']]
firms.columns=['firm','empresa','status', 'industria']

dataf = pd.merge(dataf, firms, on='firm', how='left')
dataf.head()



,firm,q,fiscalmonth,revenue,cogs,sgae,otherincome,extraordinaryitems,finexp,incometax,...,longdebt,adjprice,originalprice,sharesoutstanding,fixedassets,year,qdate,empresa,status,industria
0,A,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,63.332411,104.0000,452000.000,NaN,2000,2000Q1,"Agilent Technologies, Inc",activo,Industrias manufactureras
1,A,2000q2,6.0,4731000.0,2421000.0,1925000.0,73000.0,0.0,0.0,161000.0,...,0.0,44.911205,73.7500,452271.967,1453000.0,2000,2000Q2,"Agilent Technologies, Inc",activo,Industrias manufactureras
2,A,2000q3,9.0,7401000.0,3790000.0,3016000.0,101000.0,0.0,0.0,244000.0,...,0.0,29.801249,48.9375,453014.579,1581000.0,2000,2000Q3,"Agilent Technologies, Inc",activo,Industrias manufactureras
3,A,2000q4,12.0,10773000.0,5522000.0,4198000.0,111000.0,0.0,0.0,407000.0,...,0.0,33.340861,54.7500,456366.381,1741000.0,2000,2000Q4,"Agilent Technologies, Inc",activo,Industrias manufactureras
4,A,2001q1,3.0,2841000.0,1449000.0,1113000.0,19000.0,-25000.0,0.0,119000.0,...,0.0,18.713509,30.7300,456769.737,1821000.0,2001,2001Q1,"Agilent Technologies, Inc",activo,Industrias manufactureras


## 1.3.1 Calculation of profitability measures

Calculate log quarterly returns (rq) and log annual returns (ry).

In [17]:
dataf.set_index(['firm','qdate'],inplace=True)
dataf['ry'] = np.log(dataf['adjprice']) - np.log(dataf.groupby(['firm'])['adjprice'].shift(4))

In [18]:

dataf['rq'] = np.log(dataf['adjprice']) - np.log(dataf.groupby(['firm'])['adjprice'].shift(1))

In [19]:
dataf.head()

q  fiscalmonth     revenue       cogs       sgae  \
firm qdate                                                           
A    2000Q1  2000q1          NaN         NaN        NaN        NaN   
     2000Q2  2000q2          6.0   4731000.0  2421000.0  1925000.0   
     2000Q3  2000q3          9.0   7401000.0  3790000.0  3016000.0   
     2000Q4  2000q4         12.0  10773000.0  5522000.0  4198000.0   
     2001Q1  2001q1          3.0   2841000.0  1449000.0  1113000.0   

             otherincome  extraordinaryitems  finexp  incometax  totalassets  \
firm qdate                                                                     
A    2000Q1          NaN                 NaN     NaN        NaN          NaN   
     2000Q2      73000.0                 0.0     0.0   161000.0    7321000.0   
     2000Q3     101000.0                 0.0     0.0   244000.0    7827000.0   
     2000Q4     111000.0                 0.0     0.0   407000.0    8425000.0   
     2001Q1      19000.0            -25000.0     0.0   119000.0    9208000.0   

             ...   adjprice  originalprice  sharesoutstanding  fixedassets  \
firm qdate   ...                                                             
A    2000Q1  ...  63.332411       104.0000         452000.000          NaN   
     2000Q2  ...  44.911205        73.7500         452271.967    1453000.0   
     2000Q3  ...  29.801249        48.9375         453014.579    1581000.0   
     2000Q4  ...  33.340861        54.7500         456366.381    1741000.0   
     2001Q1  ...  18.713509        30.7300         456769.737    1821000.0   

             year                    empresa  status  \
firm qdate                                             
A    2000Q1  2000  Agilent Technologies, Inc  activo   
     2000Q2  2000  Agilent Technologies, Inc  activo   
     2000Q3  2000  Agilent Technologies, Inc  activo   
     2000Q4  2000  Agilent Technologies, Inc  activo   
     2001Q1  2001  Agilent Technologies, Inc  activo   

                             industria        ry        rq  
firm qdate                                                  
A    2000Q1  Industrias manufactureras       NaN       NaN  
     2000Q2  Industrias manufactureras       NaN -0.343710  
     2000Q3  Industrias manufactureras       NaN -0.410137  
     2000Q4  Industrias manufactureras       NaN  0.112233  
     2001Q1  Industrias manufactureras -1.219152 -0.577538  

[5 rows x 25 columns]

Calculate operating profit (also called earnings before interest and taxes) : ebit = revenue - cogs - sgae


In [21]:
dataf['ebit'] = dataf['revenue'] - dataf['cogs'] - dataf['sgae']
dataf.head()

q  fiscalmonth     revenue       cogs       sgae  \
firm qdate                                                           
A    2000Q1  2000q1          NaN         NaN        NaN        NaN   
     2000Q2  2000q2          6.0   4731000.0  2421000.0  1925000.0   
     2000Q3  2000q3          9.0   7401000.0  3790000.0  3016000.0   
     2000Q4  2000q4         12.0  10773000.0  5522000.0  4198000.0   
     2001Q1  2001q1          3.0   2841000.0  1449000.0  1113000.0   

             otherincome  extraordinaryitems  finexp  incometax  totalassets  \
firm qdate                                                                     
A    2000Q1          NaN                 NaN     NaN        NaN          NaN   
     2000Q2      73000.0                 0.0     0.0   161000.0    7321000.0   
     2000Q3     101000.0                 0.0     0.0   244000.0    7827000.0   
     2000Q4     111000.0                 0.0     0.0   407000.0    8425000.0   
     2001Q1      19000.0            -25000.0     0.0   119000.0    9208000.0   

             ...  originalprice  sharesoutstanding  fixedassets  year  \
firm qdate   ...                                                        
A    2000Q1  ...       104.0000         452000.000          NaN  2000   
     2000Q2  ...        73.7500         452271.967    1453000.0  2000   
     2000Q3  ...        48.9375         453014.579    1581000.0  2000   
     2000Q4  ...        54.7500         456366.381    1741000.0  2000   
     2001Q1  ...        30.7300         456769.737    1821000.0  2001   

                               empresa  status                  industria  \
firm qdate                                                                  
A    2000Q1  Agilent Technologies, Inc  activo  Industrias manufactureras   
     2000Q2  Agilent Technologies, Inc  activo  Industrias manufactureras   
     2000Q3  Agilent Technologies, Inc  activo  Industrias manufactureras   
     2000Q4  Agilent Technologies, Inc  activo  Industrias manufactureras   
     2001Q1  Agilent Technologies, Inc  activo  Industrias manufactureras   

                   ry        rq       ebit  
firm qdate                                  
A    2000Q1       NaN       NaN        NaN  
     2000Q2       NaN -0.343710   385000.0  
     2000Q3       NaN -0.410137   595000.0  
     2000Q4       NaN  0.112233  1053000.0  
     2001Q1 -1.219152 -0.577538   279000.0  

[5 rows x 26 columns]

Calculate operating profit margin: opm = ebit / revenue

In [22]:
dataf['opm'] = dataf['ebit']/dataf['revenue']
dataf.head()

q  fiscalmonth     revenue       cogs       sgae  \
firm qdate                                                           
A    2000Q1  2000q1          NaN         NaN        NaN        NaN   
     2000Q2  2000q2          6.0   4731000.0  2421000.0  1925000.0   
     2000Q3  2000q3          9.0   7401000.0  3790000.0  3016000.0   
     2000Q4  2000q4         12.0  10773000.0  5522000.0  4198000.0   
     2001Q1  2001q1          3.0   2841000.0  1449000.0  1113000.0   

             otherincome  extraordinaryitems  finexp  incometax  totalassets  \
firm qdate                                                                     
A    2000Q1          NaN                 NaN     NaN        NaN          NaN   
     2000Q2      73000.0                 0.0     0.0   161000.0    7321000.0   
     2000Q3     101000.0                 0.0     0.0   244000.0    7827000.0   
     2000Q4     111000.0                 0.0     0.0   407000.0    8425000.0   
     2001Q1      19000.0            -25000.0     0.0   119000.0    9208000.0   

             ...  sharesoutstanding  fixedassets  year  \
firm qdate   ...                                         
A    2000Q1  ...         452000.000          NaN  2000   
     2000Q2  ...         452271.967    1453000.0  2000   
     2000Q3  ...         453014.579    1581000.0  2000   
     2000Q4  ...         456366.381    1741000.0  2000   
     2001Q1  ...         456769.737    1821000.0  2001   

                               empresa  status                  industria  \
firm qdate                                                                  
A    2000Q1  Agilent Technologies, Inc  activo  Industrias manufactureras   
     2000Q2  Agilent Technologies, Inc  activo  Industrias manufactureras   
     2000Q3  Agilent Technologies, Inc  activo  Industrias manufactureras   
     2000Q4  Agilent Technologies, Inc  activo  Industrias manufactureras   
     2001Q1  Agilent Technologies, Inc  activo  Industrias manufactureras   

                   ry        rq       ebit       opm  
firm qdate                                            
A    2000Q1       NaN       NaN        NaN       NaN  
     2000Q2       NaN -0.343710   385000.0  0.081378  
     2000Q3       NaN -0.410137   595000.0  0.080395  
     2000Q4       NaN  0.112233  1053000.0  0.097744  
     2001Q1 -1.219152 -0.577538   279000.0  0.098205  

[5 rows x 27 columns]

Calculate net income as: netincome = ebit + otherincome + extraordinaryitems - financial expenses - incometax

In [24]:
dataf['netincome'] = dataf['ebit'] + dataf['otherincome'] + dataf['extraordinaryitems'] - dataf['finexp'] - dataf['incometax']
dataf.head()

q  fiscalmonth     revenue       cogs       sgae  \
firm qdate                                                           
A    2000Q1  2000q1          NaN         NaN        NaN        NaN   
     2000Q2  2000q2          6.0   4731000.0  2421000.0  1925000.0   
     2000Q3  2000q3          9.0   7401000.0  3790000.0  3016000.0   
     2000Q4  2000q4         12.0  10773000.0  5522000.0  4198000.0   
     2001Q1  2001q1          3.0   2841000.0  1449000.0  1113000.0   

             otherincome  extraordinaryitems  finexp  incometax  totalassets  \
firm qdate                                                                     
A    2000Q1          NaN                 NaN     NaN        NaN          NaN   
     2000Q2      73000.0                 0.0     0.0   161000.0    7321000.0   
     2000Q3     101000.0                 0.0     0.0   244000.0    7827000.0   
     2000Q4     111000.0                 0.0     0.0   407000.0    8425000.0   
     2001Q1      19000.0            -25000.0     0.0   119000.0    9208000.0   

             ...  fixedassets  year                    empresa  status  \
firm qdate   ...                                                         
A    2000Q1  ...          NaN  2000  Agilent Technologies, Inc  activo   
     2000Q2  ...    1453000.0  2000  Agilent Technologies, Inc  activo   
     2000Q3  ...    1581000.0  2000  Agilent Technologies, Inc  activo   
     2000Q4  ...    1741000.0  2000  Agilent Technologies, Inc  activo   
     2001Q1  ...    1821000.0  2001  Agilent Technologies, Inc  activo   

                             industria        ry        rq       ebit  \
firm qdate                                                              
A    2000Q1  Industrias manufactureras       NaN       NaN        NaN   
     2000Q2  Industrias manufactureras       NaN -0.343710   385000.0   
     2000Q3  Industrias manufactureras       NaN -0.410137   595000.0   
     2000Q4  Industrias manufactureras       NaN  0.112233  1053000.0   
     2001Q1  Industrias manufactureras -1.219152 -0.577538   279000.0   

                  opm  netincome  
firm qdate                        
A    2000Q1       NaN        NaN  
     2000Q2  0.081378   297000.0  
     2000Q3  0.080395   452000.0  
     2000Q4  0.097744   757000.0  
     2001Q1  0.098205   154000.0  

[5 rows x 28 columns]

Calculate profit margin (ratio) as: pm = ni / revenue

In [25]:
dataf['pm'] = dataf['netincome']/dataf['revenue']
dataf.head()

q  fiscalmonth     revenue       cogs       sgae  \
firm qdate                                                           
A    2000Q1  2000q1          NaN         NaN        NaN        NaN   
     2000Q2  2000q2          6.0   4731000.0  2421000.0  1925000.0   
     2000Q3  2000q3          9.0   7401000.0  3790000.0  3016000.0   
     2000Q4  2000q4         12.0  10773000.0  5522000.0  4198000.0   
     2001Q1  2001q1          3.0   2841000.0  1449000.0  1113000.0   

             otherincome  extraordinaryitems  finexp  incometax  totalassets  \
firm qdate                                                                     
A    2000Q1          NaN                 NaN     NaN        NaN          NaN   
     2000Q2      73000.0                 0.0     0.0   161000.0    7321000.0   
     2000Q3     101000.0                 0.0     0.0   244000.0    7827000.0   
     2000Q4     111000.0                 0.0     0.0   407000.0    8425000.0   
     2001Q1      19000.0            -25000.0     0.0   119000.0    9208000.0   

             ...  year                    empresa  status  \
firm qdate   ...                                            
A    2000Q1  ...  2000  Agilent Technologies, Inc  activo   
     2000Q2  ...  2000  Agilent Technologies, Inc  activo   
     2000Q3  ...  2000  Agilent Technologies, Inc  activo   
     2000Q4  ...  2000  Agilent Technologies, Inc  activo   
     2001Q1  ...  2001  Agilent Technologies, Inc  activo   

                             industria        ry        rq       ebit  \
firm qdate                                                              
A    2000Q1  Industrias manufactureras       NaN       NaN        NaN   
     2000Q2  Industrias manufactureras       NaN -0.343710   385000.0   
     2000Q3  Industrias manufactureras       NaN -0.410137   595000.0   
     2000Q4  Industrias manufactureras       NaN  0.112233  1053000.0   
     2001Q1  Industrias manufactureras -1.219152 -0.577538   279000.0   

                  opm  netincome        pm  
firm qdate                                  
A    2000Q1       NaN        NaN       NaN  
     2000Q2  0.081378   297000.0  0.062777  
     2000Q3  0.080395   452000.0  0.061073  
     2000Q4  0.097744   757000.0  0.070268  
     2001Q1  0.098205   154000.0  0.054206  

[5 rows x 29 columns]

Calculate asset turn over ratio: ato = revenue / totalassets

In [26]:
dataf['ato'] = dataf['revenue']/dataf['totalassets']

Calculate acid ratio: acidratio = currentassets / currentliabilities

Calculate financial leverage ratio as: finlev=longdebt / totalassets

Calculate market cap

In [32]:
#marketcap = adjprice * dataf.shareastounding
dataf['mcap'] = dataf['originalprice'] * dataf['sharesoutstanding']

dataf.head()

q  fiscalmonth     revenue       cogs       sgae  \
firm qdate                                                           
A    2000Q1  2000q1          NaN         NaN        NaN        NaN   
     2000Q2  2000q2          6.0   4731000.0  2421000.0  1925000.0   
     2000Q3  2000q3          9.0   7401000.0  3790000.0  3016000.0   
     2000Q4  2000q4         12.0  10773000.0  5522000.0  4198000.0   
     2001Q1  2001q1          3.0   2841000.0  1449000.0  1113000.0   

             otherincome  extraordinaryitems  finexp  incometax  totalassets  \
firm qdate                                                                     
A    2000Q1          NaN                 NaN     NaN        NaN          NaN   
     2000Q2      73000.0                 0.0     0.0   161000.0    7321000.0   
     2000Q3     101000.0                 0.0     0.0   244000.0    7827000.0   
     2000Q4     111000.0                 0.0     0.0   407000.0    8425000.0   
     2001Q1      19000.0            -25000.0     0.0   119000.0    9208000.0   

             ...  status                  industria        ry        rq  \
firm qdate   ...                                                          
A    2000Q1  ...  activo  Industrias manufactureras       NaN       NaN   
     2000Q2  ...  activo  Industrias manufactureras       NaN -0.343710   
     2000Q3  ...  activo  Industrias manufactureras       NaN -0.410137   
     2000Q4  ...  activo  Industrias manufactureras       NaN  0.112233   
     2001Q1  ...  activo  Industrias manufactureras -1.219152 -0.577538   

                  ebit       opm  netincome        pm       ato          mcap  
firm qdate                                                                     
A    2000Q1        NaN       NaN        NaN       NaN       NaN  4.700800e+07  
     2000Q2   385000.0  0.081378   297000.0  0.062777  0.646223  3.335506e+07  
     2000Q3   595000.0  0.080395   452000.0  0.061073  0.945573  2.216940e+07  
     2000Q4  1053000.0  0.097744   757000.0  0.070268  1.278694  2.498606e+07  
     2001Q1   279000.0  0.098205   154000.0  0.054206  0.308536  1.403653e+07  

[5 rows x 31 columns]